<a href="https://colab.research.google.com/github/riparino/ATTACK-Python-Client/blob/master/MITRE_ATT%26CK_STIX_TAXII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TAXII Servers and Intel

## Introduction to TAXII

### Installation of dependent packages

In [0]:
pip install stix2 attackcti

Trusted Automated Exchange of Intelligence Information (TAXII™) is an application protocol for exchanging CTI over HTTPS. ​TAXII defines a RESTful API (a set of services and message exchanges) and a set of requirements for TAXII Clients and Servers. As depicted below, TAXII defines two primary services to support a variety of common sharing models:

*   **Collection** - A Collection is an interface to a logical repository of CTI objects provided by a TAXII Server that allows a producer to host a set of CTI data that can be requested by consumers: TAXII Clients and Servers exchange information in a request-response model.

*   **Channel** - Maintained by a TAXII Server, a Channel allows producers to push data to many consumers and consumers to receive data from many producers: TAXII Clients exchange information with other TAXII Clients in a publish-subscribe model. *Note: The TAXII 2.0 specification reserves the keywords required for Channels but does not specify Channel services. Channels and their services will be defined in a later version of TAXII.*

Collections and Channels can be organized in different ways. For example, they can be grouped to support the needs of a particular trust group.

A TAXII server instance can support one or more API Roots. API Roots are logical groupings of TAXII Channels and Collections and can be thought of as instances of the TAXII API available at different URLs, where each API Root is the “root” URL of that particular instance of the TAXII API.

TAXII relies on existing protocols when possible. In particular, TAXII Servers are discovered within a network via DNS Service records (and/or by a Discovery Endpoint, described in the next section). In addition, TAXII uses HTTPS as the transport for all communications, and it uses HTTP for content negotiation and authentication.

TAXII was specifically designed to support the exchange of CTI represented in STIX, and support for exchanging STIX 2.0 content is mandatory to implement. However, TAXII can also be used to share data in other formats. It is important to note that STIX and TAXII are independent standards: the structures and serializations of STIX do not rely on any specific transport mechanism, and TAXII can be used to transport non-STIX data.

TAXII design principles include minimizing operational changes needed for adoption; easy integration with existing sharing agreements, and support for all widely used threat sharing models: hub-and-spoke, peer-to-peer, source-subscriber.

### Show all ATT&CK technology domains

In [6]:
from stix2 import TAXIICollectionSource
from taxii2client import Server

# Instantiate server and get API Root
server = Server("https://cti-taxii.mitre.org/taxii/")
api_root = server.api_roots[0]

# Print name and ID of all ATT&CK technology-domains available as collections
for collection in api_root.collections:
          print(collection.title + ": " + collection.id)

Enterprise ATT&CK: 95ecc380-afe9-11e4-9b6c-751b66dd541e
PRE-ATT&CK: 062767bd-02d2-4b72-84ba-56caef0f8658
Mobile ATT&CK: 2f669986-b40b-4423-b720-4396ca6a462b


The ID of each collection can then be used to get the content of that collection. Here's an example of using Enterprise ATT&CK’s ID to get that content. You'll see that changing the ID in the URL, which is highlighted in the code, allows you to get the content from another specified domain, such as ATT&CK for Mobile or PRE-ATT&CK.

In [7]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client import Collection

# Initialize dictionary to hold Enterprise ATT&CK content
attack = {}

# Establish TAXII2 Collection instance for Enterprise ATT&CK collection
collection = Collection("https://cti-taxii.mitre.org/stix/collections/95ecc380-afe9-11e4-9b6c-751b66dd541e/")

# Supply the collection to TAXIICollection
tc_source = TAXIICollectionSource(collection)

# Create filters to retrieve content from Enterprise ATT&CK based on type
filter_objs = {"techniques": Filter("type", "=", "attack-pattern"),
          "mitigations": Filter("type", "=", "course-of-action"),
          "groups": Filter("type", "=", "intrusion-set"),
          "malware": Filter("type", "=", "malware"),
          "tools": Filter("type", "=", "tool"),
          "relationships": Filter("type", "=", "relationship")
}

# Retrieve all Enterprise ATT&CK content
for key in filter_objs:
          attack[key] = tc_source.query(filter_objs[key])

# For visual purposes, print the first technique received from the server
print(attack["techniques"][0])

{
    "type": "attack-pattern",
    "id": "attack-pattern--b24e2a20-3b3d-4bf0-823b-1ed765398fb0",
    "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5",
    "created": "2019-10-09T18:48:31.906Z",
    "modified": "2019-10-14T23:29:24.908Z",
    "name": "Account Access Removal",
    "description": "Adversaries may interrupt availability of system and network resources by inhibiting access to accounts utilized by legitimate users. Accounts may be deleted, locked, or manipulated (ex: changed credentials) to remove access to accounts.\n\nAdversaries may also subsequently log off and/or reboot boxes to set malicious changes into place.(Citation: CarbonBlack LockerGoga 2019)(Citation: Unit42 LockerGoga 2019)",
    "kill_chain_phases": [
        {
            "kill_chain_name": "mitre-attack",
            "phase_name": "impact"
        }
    ],
    "external_references": [
        {
            "source_name": "mitre-attack",
            "url": "https://attack.mitre.org/techniq

## Data Mappings for ATT&CK TAXII and STIX-JSON

### Objects

In ATT&CK, there are three main concepts (excluding Tactics for now): Techniques, Groups, and Software. Most techniques also have Mitigations. STIX 2.0 describes these as objects and uses different terminology to describe them. The following table is a mapping of ATT&CK concepts to STIX 2.0 objects:


* Tactic > `x-mitre-tactic`
* Technique > `attack-pattern`
* Group > `intrusion-set`
* Software > `malware` `||` `tool`
* Mitigation > `course-of-action`
* Matrix > `x-mitre-matrix`

The above STIX types are found as literal strings assigned to the type property of the STIX JSON object. As shown in the table, in STIX 2.0, there are objects called "Course(s) of Action" used to describe mitigations to ATT&CK techniques. Similarly, the STIX 2.0 object called "Attack Pattern" describes techniques, etc. It should also be noted that Tactics are not an explicit object type in STIX 2.0, and they are referenced implicitly as kill chain phases within the other object types, as described in the tables below.

### Properties

The following is a table mapping of ATT&CK properties, the old ATT&CK MediaWiki names, and the new STIX properties. Some of these properties are standard STIX properties, while others were custom-created for compatibility with ATT&CK. These properties are accessed from STIX objects as JSON properties:

*NOTE: For simplicity's sake, I am appending the prefix of the data type (ie. `data_property.property_name`)*

#### Common

* Common.Entry ID - `external_references[i].external_id` where `external_references[i].source_name == "mitre-attack"`
* Common.Entry URL - `external_references[i].url` where `external_references[i].source_name == "mitre-attack"`
* Common.Entry Title - `name`
* Common.Entry Text - `description`
* Common.Citation - `external_references`
* Common.Deprecated - `x_mitre_deprecated`
* Common.Revoked - `revoked`

#### Techniques

* Techniques.Entry Title - `name`
* Techniques.Tactic - `kill_chain_phases[i].phase_name` where `kill_chain_phases[i].kill_chain_name == "mitre-attack"`
* Techniques.Description - `description`
* Techniques.Mitigation - `relationship` where `relationship_type == "mitigates"`, points from a source object with `type=="course-of-action"`, which contains a description
* Techniques.Detection - `description` (inline heading of "Detection")
* Techniques.Examples - `relationship`, points from the `attack-pattern` to and from `malware`, `tool`, and `intrusion-set`
* Techniques.Platform - `x_mitre_platforms`
* Techniques.Data Sources - `x_mitre_data_sources`
* Techniques.Permissions Required - `x_mitre_permissions_required`
* Techniques.Effective Permissions - `x_mitre_effective_permissions`
* Techniques.Defense Bypassed - `x_mitre_defense_bypassed`
* Techniques.System Requirements - `x_mitre_system_requirements`
* Techniques.Network Requirements - `x_mitre_network_requirements`
* Techniques.Remote Support - `x_mitre_remote_support`
* Techniques.Contributors - `x_mitre_contributors`
* Techniques.Impact Type - `x_mitre_impact_type`

#### Software

* Software.Techniques Used - `relationship` where `relationship_type == "uses"`, points to a target object with `type == "attack-pattern"`